In [51]:
import pandas as pd
import datetime
import seaborn as sns
from scipy import stats
import numpy as np
import random

In [52]:
filename = "./../data/311_Cases.csv"

In [68]:
data = pd.read_csv(filename, encoding='utf8',engine='python').\
      dropna(subset=["Neighborhood", "Category", 'Police District', 'Responsible Agency', 'Source'])

In [69]:
# quick inspection
print(len(data)) # 539,576 (after dropping NAs) # 2,924,378 # 3,362,678 (withou dropping NAs)
data.head()

2924378


,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL
0,241459,07/02/2008 03:16:55 PM,07/12/2010 10:16:59 AM,07/12/2010 10:16:59 AM,Closed,NaN,DPW Ops Queue,Tree Maintenance,Trees - Damaging_Property,Lifted_sidewalk_tree_roots,Intersection of RETIRO WAY and RICO WAY,2.0,Marina,NORTHERN,37.80493,-122.4381,"(37.8049295544814, -122.438141782293)",Phone,NaN
1,241538,07/02/2008 04:47:08 PM,06/25/2010 11:16:56 AM,06/25/2010 11:16:56 AM,Closed,NaN,DPW Ops Queue,Tree Maintenance,Trees - Damaged_Tree,Hanging_limb,Intersection of ELLIS ST and WEBSTER ST,5.0,Western Addition,NORTHERN,37.78257,-122.4308,"(37.7825698284387, -122.430797788991)",Phone,NaN
2,241571,07/02/2008 05:54:07 PM,10/21/2010 12:09:32 PM,10/21/2010 12:09:32 PM,Closed,NaN,DPW Ops Queue,Tree Maintenance,Trees - Overgrown_Tree,Blocking_street_lights,"4228 GEARY BLVD, SAN FRANCISCO, CA, 94118",1.0,Inner Richmond,RICHMOND,37.78120,-122.4650,"(37.7812, -122.465)",Phone,NaN
3,241909,07/03/2008 12:38:04 PM,06/25/2010 11:16:56 AM,06/25/2010 11:16:56 AM,Closed,NaN,DPW Ops Queue,Tree Maintenance,Trees - Landscaping,Sprinkler_system_issues,Intersection of 29TH ST and DOLORES ST,8.0,Noe Valley,INGLESIDE,37.74384,-122.4244,"(37.743838109851, -122.424384904295)",Phone,NaN
4,241917,07/03/2008 12:44:26 PM,07/06/2010 06:16:58 PM,07/06/2010 06:16:58 PM,Closed,NaN,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,Intersection of BRANNAN ST and DELANCEY ST,6.0,South Beach,SOUTHERN,37.78361,-122.3898,"(37.7836091422541, -122.389846394555)",Phone,NaN


In [55]:
# set random seed
random.seed(42)
small_data = data.sample(n = int(len(data) * 1.0)) # if needed to reduce, change to < 1.0
len(small_data) # 539,576

539576

# Select relevant variables

In [70]:
# create ClosingTime (target variable)
timedelta = pd.to_datetime(small_data["Closed"]) - pd.to_datetime(small_data["Opened"])

In [71]:
small_data['Closing Time'] = timedelta.dt.days
for_csv =  small_data[['Closing Time', "Neighborhood", "Category", 'Police District', 'Responsible Agency', 'Source']]

# filter negatives
for_csv[for_csv['Closing Time'] >= 0].sort_values(by=['Closing Time']) # 530,822 (98% of data is kept)

,Closing Time,Neighborhood,Category,Police District,Responsible Agency,Source
2196963,0,Mission,Street and Sidewalk Cleaning,MISSION,Recology_Abandoned,Mobile/Open311
1792860,0,Duboce Triangle,Graffiti,PARK,DPW Ops Queue,Mobile/Open311
1794084,0,Portola,Street and Sidewalk Cleaning,BAYVIEW,Recology_Abandoned,Mobile/Open311
2242104,0,Alamo Square,Street and Sidewalk Cleaning,NORTHERN,DPW Ops Queue,Mobile/Open311
2524031,0,South of Market,Street Defects,SOUTHERN,311 Supervisor Queue,Web
1656578,0,Mission,Graffiti,MISSION,DPW Ops Queue,Mobile/Open311
2531458,0,South of Market,Encampments,SOUTHERN,DPW Ops Queue,Mobile/Open311
1542882,0,Mission,Street and Sidewalk Cleaning,MISSION,Recology_Abandoned,Mobile/Open311
3035647,0,Mission,Encampments,MISSION,DPW Ops Queue,Mobile/Open311
3270004,0,Tenderloin,Encampments,NORTHERN,311 Supervisor Queue,Web


# Save to CSV

In [72]:
for_csv.to_csv("./../data/311_Cases_small.csv", header=False, index=False) # saves to local disk